<a href="https://colab.research.google.com/github/Yash0411/Tic-Tac-Toe-using-ML/blob/master/AI_Tic_Tac_Toe_Approach2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras import Sequential
import tensorflow as tf
import pandas as pd
import numpy as np
import random
from keras.utils.np_utils import to_categorical

In [ ]:
def initBoard():
  board = [
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]
      ]
  return board


# Print the current state of the board
def printBoard(board):
  for i in range(len(board)):
    for j in range(len(board[i])):
          mark = ' '
          if board[i][j] == 1:
              mark = 'X'
          elif board[i][j] == 2:
              mark = 'O'
          if (j == len(board[i]) - 1):
              print(mark)
          else:
              print(str(mark) + "|", end='')
    
    if (i < len(board) - 1):
          print("-----")

In [ ]:
def getAvailableMoves(board):
  am = []
  for i in range(len(board)):
    for j in range(len(board)):
      if board[i][j]==0:
        am.append((i,j))
  return am


In [ ]:
def getWinner(board):

  #Cols
  for i in range(3):
    if board[i][0]==board[i][1]==board[i][2]==1:
      return 1
    if board[i][0]==board[i][1]==board[i][2]==2:
      return 2

  #Rows
  for i in range(3):
    if board[0][i]==board[1][i]==board[2][i]==1:
      return 1
    if board[0][i]==board[1][i]==board[2][i]==2:
      return 2

  if board[0][0]==board[1][1]==board[2][2]==1 or board[0][0]==board[1][1]==board[2][2]==1:
    return 1

  if board[0][2]==board[1][1]==board[2][0]==2 or board[0][2]==board[1][1]==board[2][0]==2:
    return 2

  step=9
  for i in range(len(board)):
    for j in range(len(board)):
      if board[i][j]==0:
        step=0
        break

  if step==9:
    return 0
    
  return -1

In [ ]:
# Get best next move for the given player at the given board position
def bestMove(board, model, player, rnd=0):
  scores = []
  moves = getAvailableMoves(board)
          
  # Make predictions for each possible move
  for i in range(len(moves)):
    future = np.array(board)
    future[moves[i][0]][moves[i][1]] = player
    prediction = model.predict(future.reshape((-1, 9)))[0]
    if player == 1:
      winPrediction = prediction[1]
      lossPrediction = prediction[2]
    else:
      winPrediction = prediction[2]
      lossPrediction = prediction[1]
      drawPrediction = prediction[0]
    if winPrediction - lossPrediction > 0:
      scores.append(winPrediction - lossPrediction)
    else:
      scores.append(drawPrediction - lossPrediction)

    # Choose the best move with a random factor
    bestMoves = np.flip(np.argsort(scores))
    for i in range(len(bestMoves)):
      if random.random() * rnd < 0.5:
        return moves[bestMoves[i]]

    # Choose a move completely at random
    return moves[random.randint(0, len(moves) - 1)]

In [ ]:
def make_a_move(playerToMove,move,board):
  board[move[0]][move[1]]=playerToMove
  return board

In [ ]:
# Simulate a game
def simulateGame(p1=None, p2=None, rnd=0):
      hist = []
      board = initBoard()
      playerToMove = 1
          
      while getWinner(board) == -1:
              
        # Chose a move (random or use a player model if provided)
        move = None
        if playerToMove == 1 and p1 != None:
          move = bestMove(board, p1, playerToMove, rnd)
        elif playerToMove == 2 and p2 != None:
          move = bestMove(board, p2, playerToMove, rnd)
        else:
          moves = getAvailableMoves(board)
          move = moves[random.randint(0, len(moves) - 1)]
              
        # Make the move
        board = make_a_move(playerToMove,move,board)
              
        # Add the move to the history
        hist.append((playerToMove, move))
              
        # Switch the active player
        playerToMove = 1 if playerToMove == 2 else 2
              
      return hist

# Simulate a game
hist = simulateGame()
print(hist)

In [ ]:
#Simulate 10000 games
games = [simulateGame() for _ in range(10000)]

In [ ]:
# Reconstruct the board from the move list
def movesToBoard(moves):
  board = initBoard()
  for move in moves:
    player = move[0]
    coords = move[1]
    board[coords[0]][coords[1]] = player
  return board 

In [ ]:
# Aggregate win/loss/draw stats for a player
def gameStats(games, player=1):
  stats = {"win": 0, "loss": 0, "draw": 0}
  for game in games:
    result = getWinner(movesToBoard(game))
    if result == -1:
      continue
    elif result == player:
      stats["win"] += 1
    elif result == 0:
      stats["draw"] += 1
    else:
      stats["loss"] += 1
        
    winPct = stats["win"] / len(games) * 100
    lossPct = stats["loss"] / len(games) * 100
    drawPct = stats["draw"] / len(games) * 100

    print("Results for player %d:" % (player))
    print("Wins: %d (%.1f%%)" % (stats["win"], winPct))
    print("Loss: %d (%.1f%%)" % (stats["loss"], lossPct))
    print("Draw: %d (%.1f%%)" % (stats["draw"], drawPct))

In [ ]:
# Get a set of board states labelled by who eventually won that game
def gamesToWinLossData(games):
        X = []
        y = []
        for game in games:
            winner = getWinner(movesToBoard(game))
            for move in range(len(game)):
                X.append(movesToBoard(game[:(move + 1)]))
                y.append(winner)

        X = np.array(X).reshape((-1, 9))
        print(y[:100])
        y = to_categorical(y)
        #print(y[:10])
        # Return an appropriate train/test split
        trainNum = int(len(X) * 0.8)
        return (X[:trainNum], X[trainNum:], y[:trainNum], y[trainNum:])
    

In [ ]:
# Split out train and validation data
X_train, X_test, y_train, y_test = gamesToWinLossData(games)

#Define Model

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(200, activation='relu', input_shape=(9,)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(125, activation='relu'),
  tf.keras.layers.Dense(75, activation='relu'),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(25, activation='relu'),      
  tf.keras.layers.Dense(3, activation='softmax'),                            
])
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

#Fit Model

In [ ]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100, 
                    batch_size=100)

#Play

In [ ]:
rows = 3
cols = 3
board_size = rows * cols

class state:
  data = np.zeros((3,3))
  winner = None
  end = None
  turn = None
  
  
  def print_grid(self):
      for i in range(len(self.data)):
          for j in range(len(self.data[i])):
              mark = ' '
              if self.data[i][j] == 1:
                  mark = 'X'
              elif self.data[i][j] == -1:
                  mark = 'O'
              if (j == len(self.data[i]) - 1):
                  print(mark)
              else:
                  print(str(mark) + "|", end='')
          if (i < len(self.data) - 1):
              print("-----")
    

  def initiate(self):

    self.winner = None
    self.data = np.zeros((3,3)) 
    self.end = False
    self.turn = None


    decide = int(input("Press : \n 1. Computer First \n 2. Human First \n "))
    if decide == 1 :
      self.turn = 1
      self.play()
    elif decide == 2 :
      self.turn = 0
      self.play()
    else:
      self.initiate()

  
  def endgame(self):
    #Cols
      for i in range(3):
        if self.data[i][0]==self.data[i][1]==self.data[i][2]==1:
          return 1
        if self.data[i][0]==self.data[i][1]==self.data[i][2]==2:
          return 2

      #Rows
      for i in range(3):
        if self.data[0][i]==self.data[1][i]==self.data[2][i]==1:
          return 1
        if self.data[0][i]==self.data[1][i]==self.data[2][i]==-1:
          return 2

      if self.data[0][0]==self.data[1][1]==self.data[2][2]==1 or self.data[0][0]==self.data[1][1]==self.data[2][2]==1:
        return 1

      if self.data[0][2]==self.data[1][1]==self.data[2][0]==-1 or self.data[0][2]==self.data[1][1]==self.data[2][0]==-1:
        return 2

      step=9
      for i in range(len(self.data)):
        for j in range(len(self.data)):
          if self.data[i][j]==0:
            step=0
            break
      if step==9:
        return 0
      return -1



    # Print the current state of the board
  def printBoard(self):
      for i in range(len(self.data)):
          for j in range(len(self.data[i])):
              mark = ' '
              if board[i][j] == 1:
                  mark = 'X'
              elif board[i][j] == 2:
                  mark = 'O'
              if (j == len(board[i]) - 1):
                  print(mark)
              else:
                  print(str(mark) + "|", end='')
          if (i < len(self.data) - 1):
              print("-----")


  def play(self): 
    step = 0

    while self.end!=1 and step<9:
    
      if self.turn == 1 :
        self.make_a_move()
    
      elif self.turn == 0 :
        self.human_to_move()
      
      step+=1

      #Check Winner ar is a Tie
      if self.endgame()==1:
        print("AI wins")
        break

      if self.endgame()==2:
        print("Player wins")
        break

      if self.endgame()==0:
        print("Draw")
        break


  def human_to_move(self):
    x,y = map(int, input("Enter x , y : ").split())
    if self.data[x-1][y-1]==0:
      print("\nYour Turn : \n")
      self.data[x-1][y-1] = -1
      self.turn = 1
      self.print_grid()
    else : 
      print("Enter proper value")
      self.human_to_move()


  def make_a_move(self):
    print("\nComputer's Turn : \n")
    k = bestMove(self.data, model, 1)
    print(k)
    #l=model.predict(k)
    x = k[0]
    y = k[1]
    self.data[x][y] = 1
    self.turn = 0
    self.print_grid()

      
    
s1 = state()
s1.initiate()

# ALL the above code at a glance.

In [ ]:
import csv
import numpy as np
import pandas as pd
import os
import random
import copy

# name of csv file  
filename = "/content/dataset.csv"
fields = []
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    #csvwriter.writerow(fields)  

    rows = 3
    cols = 3
    board_size = rows * cols

    
    winner = None
    turn = None
    gameHist = []
    game = []

    X = 0
    O = 0
    D = 0

    def initBoard():
      board = [
          [0, 0, 0],
          [0, 0, 0],
          [0, 0, 0]
      ]
      return board


    # Print the current state of the board
    def printBoard(board):
      for i in range(len(board)):
          for j in range(len(board[i])):
              mark = ' '
              if board[i][j] == 1:
                  mark = 'X'
              elif board[i][j] == 2:
                  mark = 'O'
              if (j == len(board[i]) - 1):
                  print(mark)
              else:
                  print(str(mark) + "|", end='')
          if (i < len(board) - 1):
              print("-----")
    

    def getAvailableMoves(board):
      am = []
      for i in range(len(board)):
        for j in range(len(board)):
          if board[i][j]==0:
            am.append((i,j))
      return am

    
    def getWinner(board):

      #Cols
      for i in range(3):
        if board[i][0]==board[i][1]==board[i][2]==1:
          return 1
        if board[i][0]==board[i][1]==board[i][2]==2:
          return 2

      #Rows
      for i in range(3):
        if board[0][i]==board[1][i]==board[2][i]==1:
          return 1
        if board[0][i]==board[1][i]==board[2][i]==2:
          return 2

      if board[0][0]==board[1][1]==board[2][2]==1 or board[0][0]==board[1][1]==board[2][2]==1:
        return 1

      if board[0][2]==board[1][1]==board[2][0]==2 or board[0][2]==board[1][1]==board[2][0]==2:
        return 2

      step=9
      for i in range(len(board)):
        for j in range(len(board)):
          if board[i][j]==0:
            step=0
            break
      if step==9:
        return 0
      return -1


    # Get best next move for the given player at the given board position
    def bestMove(board, model, player, rnd=0):
      scores = []
      moves = getAvailableMoves(board)
          
      # Make predictions for each possible move
      for i in range(len(moves)):
        future = np.array(board)
        future[moves[i][0]][moves[i][1]] = player
        prediction = model.predict(future.reshape((-1, 9)))[0]
        if player == 1:
          winPrediction = prediction[1]
          lossPrediction = prediction[2]
        else:
          winPrediction = prediction[2]
          lossPrediction = prediction[1]
          drawPrediction = prediction[0]
        if winPrediction - lossPrediction > 0:
          scores.append(winPrediction - lossPrediction)
        else:
          scores.append(drawPrediction - lossPrediction)

        # Choose the best move with a random factor
        bestMoves = np.flip(np.argsort(scores))
        for i in range(len(bestMoves)):
          if random.random() * rnd < 0.5:
            return moves[bestMoves[i]]

        # Choose a move completely at random
        return moves[random.randint(0, len(moves) - 1)]

      
    def make_a_move(playerToMove,move,board):
      board[move[0]][move[1]]=playerToMove
      return board


    # Simulate a game
    def simulateGame(p1=None, p2=None, rnd=0):
      hist = []
      board = initBoard()
      playerToMove = 1
          
      while getWinner(board) == -1:
              
        # Chose a move (random or use a player model if provided)
        move = None
        if playerToMove == 1 and p1 != None:
          move = bestMove(board, p1, playerToMove, rnd)
        elif playerToMove == 2 and p2 != None:
          move = bestMove(board, p2, playerToMove, rnd)
        else:
          moves = getAvailableMoves(board)
          move = moves[random.randint(0, len(moves) - 1)]
              
        # Make the move
        board = make_a_move(playerToMove,move,board)
              
        # Add the move to the history
        hist.append((playerToMove, move))
              
        # Switch the active player
        playerToMove = 1 if playerToMove == 2 else 2
              
      return hist

    # Simulate a game
    hist = simulateGame()
    print(hist)

    games = [simulateGame() for _ in range(10000)]


   # Reconstruct the board from the move list
    def movesToBoard(moves):
        board = initBoard()
        for move in moves:
            player = move[0]
            coords = move[1]
            board[coords[0]][coords[1]] = player
        return board 



    # Aggregate win/loss/draw stats for a player
    def gameStats(games, player=1):
        stats = {"win": 0, "loss": 0, "draw": 0}
        for game in games:
            result = getWinner(movesToBoard(game))
            if result == -1:
                continue
            elif result == player:
                stats["win"] += 1
            elif result == 0:
                stats["draw"] += 1
            else:
                stats["loss"] += 1
        
        winPct = stats["win"] / len(games) * 100
        lossPct = stats["loss"] / len(games) * 100
        drawPct = stats["draw"] / len(games) * 100

        print("Results for player %d:" % (player))
        print("Wins: %d (%.1f%%)" % (stats["win"], winPct))
        print("Loss: %d (%.1f%%)" % (stats["loss"], lossPct))
        print("Draw: %d (%.1f%%)" % (stats["draw"], drawPct))


    # Get a set of board states labelled by who eventually won that game
    def gamesToWinLossData(games):
        X = []
        y = []
        for game in games:
            winner = getWinner(movesToBoard(game))
            for move in range(len(game)):
                X.append(movesToBoard(game[:(move + 1)]))
                y.append(winner)

        X = np.array(X).reshape((-1, 9))
        print(y[:10])
        y = to_categorical(y)
        print(y[:10])
        # Return an appropriate train/test split
        trainNum = int(len(X) * 0.8)
        return (X[:trainNum], X[trainNum:], y[:trainNum], y[trainNum:])
    

    csvwriter.writerows(games)  
    # Split out train and validation data
    X_train, X_test, y_train, y_test = gamesToWinLossData(games)
        

